In [115]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms
import matplotlib.pyplot as plt
import numpy as np
import glob
# import pandas as pd
from torch.utils.data import DataLoader

In [116]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

data = torchvision.datasets.ImageFolder(
                    root='./data/trainset/',
                    transform=transforms
                    )

test_data = torchvision.datasets.ImageFolder(
                    root= './data/test/', 
                    transform=transforms)

dataset_ratio = np.array([95, 5])/100

sizes = [int(x*len(data)) for x in dataset_ratio]
sizes[0] += len(data) - sum(sizes)

train_dataset, valid_dataset = torch.utils.data.random_split(dataset=data, lengths=sizes)

train_loader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

valid_loader = torch.utils.data.DataLoader(
                    valid_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

test_loader = torch.utils.data.DataLoader(
                    test_data,
                    batch_size=64,
                    num_workers=2,
                    shuffle=False
                    )
print(data)
print(valid_dataset)

In [3]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


## Hyper-parameters of the model

In [ ]:
learning_rate = 1e-3
lr0 = 0.002
optim = 'adam'
num_epochs = 10
store_every = 200

## Defining a residual convolution block to be used in the models

In [5]:
class ResidualConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3, 3), padding=1, activation = nn.ReLU):
        super(ResidualConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.activation = activation
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size)
        self.relu1 = nn.ReLU(in_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size)
        self.relu2 = nn.ReLU(in_channels)
        
        if in_channels != out_channels:
#             self.projected_conv = self.conv1x1(in_channels, out_channels)
            self.project_linear = nn.Linear(in_channels, out_channels)
    def forward(x):
        identity = x

        out = self.conv1(x)
        out = self.relu1(out)

        out = self.conv2(out)
        if self.in_channels == self.out_channels:
            out += identity
        else:
            out += self.project_linear(identity)
        out = self.relu2(out)
        
    def conv1x1(in_channels, out_channels, stride=1):
        """1x1 convolution"""
        print()
        return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

In [6]:
class ResidualClassifier(nn.Module):
    def __init__(self):
        super(ResidualClassifier, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            ResidualConv(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            ResidualConv(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            ResidualConv(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            )
        # Logistic Regression
        self.clf = nn.Linear(64, 2)
        
        def forward(x):
            return self.clf(self.conv(x).squeeze())

## Vanila CNN

In [117]:
class VanilaCNN(nn.Module):
    """Convnet Classifier"""
    def __init__(self):
        super(VanilaCNN, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 5
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 6
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(256, 2)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())

## First Implementation

In [111]:
print(model)

VanilaCNN(
  (conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1)

In [10]:
summary(model, input_size=(3,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             448
              ReLU-2           [-1, 16, 64, 64]               0
         MaxPool2d-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 32, 32, 32]           4,640
              ReLU-5           [-1, 32, 32, 32]               0
         MaxPool2d-6           [-1, 32, 16, 16]               0
            Conv2d-7           [-1, 64, 16, 16]          18,496
              ReLU-8           [-1, 64, 16, 16]               0
         MaxPool2d-9             [-1, 64, 8, 8]               0
           Conv2d-10            [-1, 128, 8, 8]          73,856
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Conv2d-13            [-1, 128, 4, 4]         147,584
             ReLU-14            [-1, 12

In [112]:
## Second implementation